In [1]:
source("helper.R")

max_expid<-get_max("Log")+1
exp_name<-"Pricing"
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")

## get logger

use_saudi<-TRUE 
seed_random<-123
n=30

futile.logger::flog.info(glue("use_saudi: {use_saudi}"), name = "log")

set.seed(seed_random)

futile.logger::flog.info(glue("use seed: {seed_random}"), name = "log")

futile.logger::flog.info(glue("number of experiments: {n}"), name = "log")

if(!use_saudi){

    data_ori <- as.data.frame(read_csv("data/IFclose.csv"))
    logpt<-log(data_ori$close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
    date <- as.Date(as.character(data_ori$Date[1:(length(data_ori$Date)-1)]), format = "%Y%m%d")
    date_timeindex_ori<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex_ori - min(date_timeindex_ori))/86400 ## 1 day = 86400 seconds
    mesh <- fm_mesh_1d(date_timeindex)
    data = data.frame(logreturn = logreturn, times = date_timeindex)
}else{
    data_ori <- as.data.frame(read_csv("data/1180_vol.csv"))
    logpt<-log(data_ori$Close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)] ## 1 day = 86400 seconds
    date <- data_ori$Date[1:(length(data_ori$Date)-1)]
    date_timeindex_ori<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex_ori - min(date_timeindex_ori))/86400
    mesh <- fm_mesh_1d(date_timeindex)
    data = data.frame(logreturn = logreturn, times = date_timeindex)
}



Loading required package: Matrix



This is INLA_25.06.07 built 2025-06-11 18:54:45 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This is rSPDE 2.5.1
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.



INFO [2025-08-07 15:56:49] start experiment
INFO [2025-08-07 15:56:49] use_saudi: TRUE
INFO [2025-08-07 15:56:49] use seed: 123
INFO [2025-08-07 15:56:49] number of experiments: 30


Rows: 2372 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): Open, High, Low, Close, Volume, Dividends, Stock Splits, vol10, vol30
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
head(data)

,logreturn,times
,<dbl>,<dbl>
1,0.091630309,0
2,0.000000000,1
3,0.184623021,4
4,-0.059188926,5
5,-0.008163110,6
6,0.004089759,7


In [3]:
set.seed(123)
best.rspde.order<-4
best.prior.range.nominal<-118.271
best.prior.std.dev.nominal<-0.7802

In [4]:

OU_model <- rspde.matern(
    mesh = mesh,
    parameterization = "matern",
    rspde.order = best.rspde.order,
    prior.range.nominal = best.prior.range.nominal,
    prior.std.dev.nominal = best.prior.std.dev.nominal
  )
    
    formula_OU <- logreturn ~  field(times, model = OU_model)
fit<-bru(formula_OU, family="stochvol", data = data)
result_fit<- rspde.result(fit, "field", OU_model, parameterization="matern")
summary(result_fit)

,mean,sd,0.025quant,0.5quant,0.975quant,mode
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
std.dev,2.292440,0.7764850,1.25719000,2.1251900,4.255350,1.8197900
range,29.339700,17.0379000,9.49637000,25.0391000,74.267900,18.7028000
nu,0.038198,0.0315074,0.00427082,0.0297026,0.121555,0.0122888


In [5]:
library(lubridate)
# 假设 date 是一个 Date 向量
#date <- as.Date(c("2025-07-10", "2025-07-15", "2025-07-20"))  # 示例数据

# 获取最后一天
last_day <- max(date)
# 生成从 next_day 开始到下个月同一天之前的所有日期
next_day <- last_day + 1
end_day <- last_day %m+% months(1)

# 生成完整序列
all_days <- seq.Date(from = next_day, to = end_day, by = "day")

# 只保留非周末（即工作日）
biz_days <- all_days[!weekdays(all_days) %in% c("Saturday", "Sunday")]
date_timeindex_new<-as.numeric(as.POSIXct(biz_days)) 
date_timeindex_new <- (date_timeindex_new - min(date_timeindex_ori))/86400 ## 1 day = 86400 seconds
# 保存到数据框
new_data <- data.frame(times = date_timeindex_new)



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [6]:
    predict(fit, new_data, ~ exp((field+Intercept)/2), n.samples = 10)

times,mean,sd,q0.025,q0.5,q0.975,median,sd.mc_std_err,mean.mc_std_err
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3418,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3419,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3422,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3423,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3424,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3425,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3426,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3429,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978
3430,0.01529209,0.005427421,0.009285709,0.01430762,0.02551451,0.01430762,0.001468367,0.002644978


In [ ]:
St=39.59## current price
r=0.02 ## risk-free rate
T=1 ## time to maturity
K=40 ## strike price
new_data = list(times = c(3419))

prediction<-predict(fit,newdata =new_data,~ c(
    "S_T" = St * exp(-r * T + 0.5 * exp(f_time) * T + sqrt(exp(f_time * T)) * Z),
    "discounted_payoff" = exp(-r * T) * pmax(S_T - K, 0)) )

In [30]:
?generate

generate               package:generics                R Documentation

_G_e_n_e_r_a_t_e _v_a_l_u_e_s _b_a_s_e_d _o_n _i_n_p_u_t_s

_D_e_s_c_r_i_p_t_i_o_n:

     Generate values based on inputs

_U_s_a_g_e:

     generate(x, ...)
     
_A_r_g_u_m_e_n_t_s:

       x: An object.

     ...: Other arguments passed to methods

_M_e_t_h_o_d_s:

     No methods found in currently loaded packages.


In [28]:

  input.df <- data.frame(x = cos(1:10))
  input.df <- within(input.df, y <- 5 + 2 * cos(1:10) + rnorm(10, mean = 0, sd = 0.1))

  # Fit the model

  fit <- bru(y ~ xeff(main = x, model = "linear"),
    family = "gaussian", data = input.df
  )
  summary(fit)

  # Generate samples for some predefined x

  df <- data.frame(x = seq(-4, 4, by = 0.1))
  smp <- generate(fit, df, ~ xeff + Intercept, n.samples = 10)

  # # Plot the resulting realizations
  # par(bg = "white") 
  # plot(df$x, smp[, 1], type = "l")
  # for (k in 2:ncol(smp)) points(df$x, smp[, k], type = "l")

  # We can also draw samples form the joint posterior

  # df <- data.frame(x = 1)
  # smp <- generate(fit, df, ~ data.frame(xeff, Intercept), n.samples = 10)
  # smp[[1]]

  # # ... and plot them
 
  # plot(do.call(rbind, smp))


inlabru version: 2.13.0
INLA version: 25.06.07
Components:
xeff: main = linear(x), group = exchangeable(1L), replicate = iid(1L), NULL
Intercept: main = linear(1), group = exchangeable(1L), replicate = iid(1L), NULL
Observation models:
  Family: 'gaussian'
    Tag: <No tag>
    Data class: 'data.frame'
    Response class: 'numeric'
    Predictor: y ~ .
    Additive/Linear: TRUE/TRUE
    Used components: effects[xeff, Intercept], latent[]
Time used:
    Pre = 0.352, Running = 0.269, Post = 0.0186, Total = 0.64 
Fixed effects:
           mean    sd 0.025quant 0.5quant 0.975quant  mode kld
xeff      1.975 0.039      1.896    1.975      2.054 1.975   0
Intercept 4.988 0.028      4.933    4.988      5.044 4.988   0

Model hyperparameters:
                                          mean    sd 0.025quant 0.5quant
Precision for the Gaussian observations 161.19 72.03      52.77   150.64
                                        0.975quant   mode
Precision for the Gaussian observations     330.67 1

In [29]:
smp

-2.93129719,-3.44628327,-3.01579931,-2.66828555,-2.71061659,-2.700277600,-3.11255614,-2.75607954,-2.95400676,-2.93928861
-2.73516388,-3.23506848,-2.81643857,-2.47647700,-2.51778653,-2.508039188,-2.90999928,-2.56170232,-2.75442723,-2.74100927
-2.53903056,-3.02385370,-2.61707783,-2.28466845,-2.32495647,-2.315800777,-2.70744242,-2.36732510,-2.55484771,-2.54272992
-2.34289724,-2.81263891,-2.41771709,-2.09285990,-2.13212641,-2.123562365,-2.50488556,-2.17294788,-2.35526818,-2.34445057
-2.14676392,-2.60142412,-2.21835635,-1.90105135,-1.93929634,-1.931323954,-2.30232870,-1.97857066,-2.15568866,-2.14617122
-1.95063060,-2.39020933,-2.01899561,-1.70924280,-1.74646628,-1.739085542,-2.09977184,-1.78419344,-1.95610913,-1.94789187
-1.75449729,-2.17899454,-1.81963487,-1.51743425,-1.55363622,-1.546847131,-1.89721499,-1.58981622,-1.75652961,-1.74961253
-1.55836397,-1.96777975,-1.62027414,-1.32562570,-1.36080615,-1.354608719,-1.69465813,-1.39543900,-1.55695008,-1.55133318
-1.36223065,-1.75656496,-1.42091340,-1.13381715,-1.16797609,-1.162370307,-1.49210127,-1.20106178,-1.35737056,-1.35305383
-1.16609733,-1.54535017,-1.22155266,-0.94200860,-0.97514603,-0.970131896,-1.28954441,-1.00668456,-1.15779103,-1.15477448
-0.96996401,-1.33413538,-1.02219192,-0.75020005,-0.78231596,-0.777893484,-1.08698755,-0.81230734,-0.95821151,-0.95649513


In [20]:
smp

xeff,Intercept
<dbl>,<dbl>
2.048166,4.981599
xeff,Intercept
<dbl>,<dbl>
2.01157,4.992041
xeff,Intercept
<dbl>,<dbl>
1.995124,4.907089
xeff,Intercept
<dbl>,<dbl>


In [ ]:
  # Plot the resulting realizations
  par(bg = "white") 
  plot(df$x, smp[, 1], type = "l")
  for (k in 2:ncol(smp)) points(df$x, smp[, k], type = "l")

  # We can also draw samples form the joint posterior

  df <- data.frame(x = 1)
  smp <- generate(fit, df, ~ data.frame(xeff, Intercept), n.samples = 10)
  smp[[1]]

  # ... and plot them
 
  plot(do.call(rbind, smp))

In [ ]:
# \donttest{
if (bru_safe_inla() &&
    require("sn", quietly = TRUE) &&
    require("ggplot2", quietly = TRUE) &&
    require("terra", quietly = TRUE) &&
    require("sf", quietly = TRUE)) {

  # Load the Gorilla data

  gorillas <- gorillas_sf

  # Plot the Gorilla nests, the mesh and the survey boundary

  ggplot() +
    gg(gorillas$mesh) +
    gg(gorillas$nests) +
    gg(gorillas$boundary, alpha = 0.1)

  # Define SPDE prior

  matern <- INLA::inla.spde2.pcmatern(
    gorillas$mesh,
    prior.sigma = c(0.1, 0.01),
    prior.range = c(0.01, 0.01)
  )

  # Define domain of the LGCP as well as the model components (spatial SPDE effect and Intercept)

  cmp <- geometry ~ field(geometry, model = matern) + Intercept(1)

  # Fit the model, with "eb" instead of full Bayes
  fit <- lgcp(
    cmp,
    data = gorillas$nests,
    samplers = gorillas$boundary,
    domain = list(geometry = gorillas$mesh),
    options = list(control.inla = list(int.strategy = "eb"))
  )

  # Once we obtain a fitted model the predict function can serve various purposes.
  # The most basic one is to determine posterior statistics of a univariate
  # random variable in the model, e.g. the intercept

  icpt <- predict(fit, NULL, ~ c(Intercept = Intercept_latent))
  plot(icpt)

  # The formula argument can take any expression that is valid within the model, for
  # instance a non-linear transformation of a random variable

  exp.icpt <- predict(fit, NULL, ~ c(
    "Intercept" = Intercept_latent,
    "exp(Intercept)" = exp(Intercept_latent)
  ))
  plot(exp.icpt, bar = TRUE)

  # The intercept is special in the sense that it does not depend on other variables
  # or covariates. However, this is not true for the smooth spatial effects 'field'.
  # In order to predict 'field' we have to define where (in space) to predict. For
  # this purpose, the second argument of the predict function can take \code{data.frame}
  # objects as well as Spatial objects. For instance, we might want to predict
  # 'field' at the locations of the mesh vertices. Using

  vrt <- fm_vertices(gorillas$mesh, format = "sf")

  # we obtain these vertices as a SpatialPointsDataFrame

  ggplot() +
    gg(gorillas$mesh) +
    gg(vrt, color = "red")

  # Predicting 'mySmooth' at these locations works as follows

  field <- predict(fit, vrt, ~field)

  # Note that just like the input also the output will be a sf object with
  # points and that the predicted statistics are simply added as columns

  class(field)
  head(vrt)
  head(field)

  # Plotting the mean, for instance, at the mesh node is straight forward

  ggplot() +
    gg(gorillas$mesh) +
    gg(field, aes(color = mean), size = 2)

  # However, we are often interested in a spatial field and thus a linear interpolation,
  # which can be achieved by using the gg mechanism for meshes

  ggplot() +
    gg(gorillas$mesh, color = field$mean)

  # Alternatively, we can predict the spatial field at a grid of locations, e.g. a
  # sf object with a grid of points covering the relevant part of mesh

  pxl <- fm_pixels(gorillas$mesh, format = "sf", mask = gorillas$boundary)
  field2 <- predict(fit, pxl, ~field)

  # This will give us a sf with the columns we are looking for

  head(field2)
  ggplot() +
    gg(gorillas$boundary) +
    gg(data = field2, geom = "tile")
}

# }


In [ ]:
exp.icpt <- predict(fit, NULL, ~ c(
"Intercept" = Intercept_latent,
"exp(Intercept)" = exp(Intercept_latent)
))
plot(exp.icpt, bar = TRUE)

In [ ]:
  icpt <- predict(fit, NULL, ~ c(Intercept = Intercept_latent))
  plot(icpt)

In [ ]:
icpt